In [1]:
import pandas as pd
import argparse
import json
import os
import configparser
from itertools import groupby

In [2]:
ion_mode = 'pos'
jobID = '123456'
tpfolder = r'/home/rbarreror/projects/TurboPutative-web/src/public/jobs/123456_pos'

# Read index
f = open(r'/home/rbarreror/projects/TurboPutative-web/src/TurboOmicsIntegrator/jobs/123456/EDA/xPreProcessing/index.json')
index = json.load(f)
f.close()

# Read cm    
xm = pd.read_json(
    r'/home/rbarreror/projects/TurboPutative-web/src/TurboOmicsIntegrator/jobs/123456/EDA/xPreProcessing/xm_norm.json'
)
xm.index = index['xm']
xm = xm.T

# Read m2i
m2i = pd.read_json(
    r'/home/rbarreror/projects/TurboPutative-web/src/TurboOmicsIntegrator/jobs/123456/EDA/xPreProcessing/m2i.json'
)
m2i.index = index['m2i']

# Read annParams
f = open(r'/home/rbarreror/projects/TurboPutative-web/src/TurboOmicsIntegrator/jobs/123456/CMM_TP/annParams.json')
annParams = json.load(f)
f.close()

# Read cmm
cmm = pd.read_json(
    r'/home/rbarreror/projects/TurboPutative-web/src/TurboOmicsIntegrator/jobs/123456/CMM_TP/pos/CMM_pos.json'
)

In [3]:
# Rename columns of cmm and write
cmm = cmm.rename(columns={
        'identifier': 'Identifier',
        'EM': 'Experimental mass',
        'adduct': 'Adduct',
        'error_ppm': 'mz Error (ppm)',
        'molecular_weight': 'Molecular Weight',
        'name': 'Name',
        'formula': 'Formula'
    })

cmm.to_csv(
    os.path.join(tpfolder, f'CMM_{ion_mode}.tsv'),
    sep='\t', index=False
)

In [4]:
# Create FeatInfo file and write
xm.columns = ['_quant_'+i for i in xm.columns]

tmtable = m2i[
    m2i[
        annParams['ionCol']['id']
        ] == annParams['ionValPos']['id'] 
        if ion_mode == 'pos' 
        else annParams['ionValNeg']['id']
]
tmtable = tmtable.join(xm, how='inner')

tmtable = tmtable.rename(
    columns={
        annParams['mzCol']['id']: 'Experimental mass'
        }
    )

if annParams['rtCol'] != None:
    tmtable = tmtable.rename(
    columns={
        annParams['rtCol']['id']: 'rt'
        }
    )

tmtable.to_csv(
    os.path.join(tpfolder, f'TM_Table_{ion_mode}.tsv'),
    sep='\t', index=False
)

In [11]:
# configUser.ini

config = configparser.ConfigParser()
config.read(os.path.join(tpfolder, 'configUser.ini'))
config['TPMetrics']['i_pattern'] = '_quant_'

config['TPMetrics']['rt1'] = str(annParams[f'mRtW{"Pos" if ion_mode=="pos" else "Neg"}'])
config['TPMetrics']['rt2'] = str(annParams[f'lRtW{"Pos" if ion_mode=="pos" else "Neg"}'])

class_adducts = annParams[f'lipidAdd{"Pos" if ion_mode=="pos" else "Neg"}']

config['TPMetrics']['class_adducts'] = str({
    cl: list(list(zip(*list(ad)))[1])
    for cl, ad in groupby(sorted(class_adducts, key=lambda x: x[0]), key=lambda x: x[0])
    }).replace("'", '"')

with open(os.path.join(tpfolder, 'configUser.ini'), 'w') as configfile:
    config.write(configfile)

In [6]:
# Pair Experimental Mass-Retention Time

if annParams['rtCol'] != None:
    tmtable[['Experimental mass', 'rt']].to_csv(
        os.path.join(tpfolder, f'mz_rt.tsv'),
        sep='\t', index=False
    )

In [15]:
tm = pd.read_csv(r'/home/rbarreror/projects/TurboPutative-web/src/public/jobs/123456_pos/tsv/4_TableMerger.tsv', sep='\t')

In [18]:
mzrt = pd.read_csv(r'/home/rbarreror/projects/TurboPutative-web/src/public/jobs/123456_pos/tsv/mz_rt.tsv', sep='\t')

In [35]:
mzrt

,Experimental mass,rt
0,760.58465,19.341
1,876.80088,23.952
2,782.56882,18.116
3,758.56897,18.518
4,806.56868,17.913
5,874.78481,23.420
6,848.76963,23.348
7,784.58393,19.020
8,810.59934,19.600
9,902.81583,23.995


In [44]:
import numpy as np

tm[[
np.logical_and(
        mzrt.iloc[:, 0]-i<0.0001, 
        mzrt['rt']-j<0.0001
    ).sum()>0
for i, j in zip(
    tm['Experimental mass'],
    tm['rt']
    )
]]

,fid,TM_Table_pos_Name,Experimental mass,rt,Platform,Mode,_quant_LD2169,_quant_LD2170,_quant_LD2171,_quant_LD2243,...,hmdb_uri,lipidmaps_compound,lipidmaps_uri,metlin_compound,metlin_uri,pubchem_compound,pubchem_uri,inChiKey,pathways,_TPIDX
0,P4,A00004,758.56897,18.518,C18,POS,-0.501769,0.637105,0.289704,0.980566,...,http://www.hmdb.ca/metabolites/HMDB0008133 ; h...,LMGP01010932 ; LMGP01010594 ; LMGP01010687 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59579.0 ; 59422.0 ; 59452.0 ; 59453.0 ; 59742....,https://metlin.scripps.edu/metabo_info.php?mol...,24778972 ; 5287971 ; 24778766 ; 24778767 ; 529...,https://pubchem.ncbi.nlm.nih.gov/compound/2477...,FWMYJLDHIVCJCT-ZTIMHPMXSA-N ; JLPULHDHAOZNQI-Z...,[],341 ; 344 ; 353 ; 354 ; 355 ; 356 ; 358 ; 359 ...
1,P4,A00004,758.56897,18.518,C18,POS,-0.501769,0.637105,0.289704,0.980566,...,NaN,LMGP03030018 ; LMGP03030039 ; LMGP03030062 // ...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,78758.0 ; 78779.0 ; 78802.0 // 78659.0 ; 78674...,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,GIONAPQXCTWOFO-AFPNGVDTSA-N ; LUACNSMEYWBEHO-A...,[],412 ; 413 ; 414 // 415 ; 416 ; 417
2,P4,A00004,758.56897,18.518,C18,POS,-0.501769,0.637105,0.289704,0.980566,...,NaN,LMGP10010764 ; LMGP10010273 ; LMGP10010302 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,81930.0 ; 81439.0 ; 81468.0 ; 81740.0 ; 81770....,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,QAWMHBUXVFPCLZ-SYGIZULESA-N ; LCSCBGYCNNFQBV-S...,[],418 ; 419 ; 420 ; 421 ; 422 ; 423 ; 424 ; 425 ...
3,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,http://www.hmdb.ca/metabolites/HMDB0008003 ; h...,LMGP01011483 ; LMGP01010576 ; LMGP01011787 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59451.0 ; 59420.0 ; 59709.0 ; 59741.0 ; 59483....,https://metlin.scripps.edu/metabo_info.php?mol...,52922448 ; 24778688 ; 52923053 ; 52923093 ; 24...,https://pubchem.ncbi.nlm.nih.gov/compound/5292...,LJUDFFWSOQEQKW-QCFYSATCSA-N ; AEPVLLKGEBHLEJ-K...,[],0 ; 1 ; 3 ; 12 ; 13 ; 15 ; 16 ; 17 ; 18 ; 19 ;...
4,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,NaN,LMGP03020070 ; LMGP03020087 ; LMGP03020091,http://www.lipidmaps.org/data/LMSDRecord.php?L...,78717.0 ; 78734.0 ; 78738.0,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,BSYSTFKERYSAHH-PVXQIPPMSA-N ; IQICQBYOIPVODO-P...,[],60 ; 61 ; 62
5,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,NaN,NaN,NaN,40767.0 ; 40769.0,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,NaN,[],63 ; 64
6,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,NaN,LMGP10010763 ; LMGP10010272 ; LMGP10010544 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,81929.0 ; 81438.0 ; 81710.0 ; 81467.0 ; 81739....,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,FKASCVIVTHYAIW-YIHXYWFXSA-N ; KMVWAJLBKVNELJ-B...,[],65 ; 66 ; 67 ; 68 ; 69 ; 70 ; 71 ; 72 ; 73 ; 7...
7,P3,A00003,782.56882,18.116,C18,POS,-0.137611,0.566504,-0.190560,-0.264693,...,http://www.hmdb.ca/metabolites/HMDB0008138 ; h...,LMGP01010937 ; LMGP01012195 ; LMGP01010955 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59584.0 ; 59811.0 ; 59649.0 ; 59680.0 ; 59875....,https://metlin.scripps.edu/metabo_info.php?mol...,0 ; 5288075 ; 53478951 ; 24778989 ; 52922901 ;...,https://pubchem.ncbi.nlm.nih.gov/compound/0 ; ...,FVXDQWZBHIXIEJ-LNDKUQBDSA-N ; VRXRAEQYIWJLSH-O...,[],190 ; 191 ; 192 ; 193 ; 194 ; 195 ; 196 ; 197 ...
8,P3,A00003,782.56882,18.116,C18,POS,-0.137611,0.566504,-0.190560,-0.264693,...,http://www.hmdb.ca/metabolites/HMDB0008462 ; h...,LMGP01010007 // LMGP02010815 ; LMGP02010560 ; ...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,39371.0 ; 39372.0 ; 39513.0 ; 39516.0 ; 39517....,https://metlin.scr

In [19]:
tm

,fid,TM_Table_pos_Name,Experimental mass,rt,Platform,Mode,_quant_LD2169,_quant_LD2170,_quant_LD2171,_quant_LD2243,...,hmdb_uri,lipidmaps_compound,lipidmaps_uri,metlin_compound,metlin_uri,pubchem_compound,pubchem_uri,inChiKey,pathways,_TPIDX
0,P4,A00004,758.56897,18.518,C18,POS,-0.501769,0.637105,0.289704,0.980566,...,http://www.hmdb.ca/metabolites/HMDB0008133 ; h...,LMGP01010932 ; LMGP01010594 ; LMGP01010687 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59579.0 ; 59422.0 ; 59452.0 ; 59453.0 ; 59742....,https://metlin.scripps.edu/metabo_info.php?mol...,24778972 ; 5287971 ; 24778766 ; 24778767 ; 529...,https://pubchem.ncbi.nlm.nih.gov/compound/2477...,FWMYJLDHIVCJCT-ZTIMHPMXSA-N ; JLPULHDHAOZNQI-Z...,[],341 ; 344 ; 353 ; 354 ; 355 ; 356 ; 358 ; 359 ...
1,P4,A00004,758.56897,18.518,C18,POS,-0.501769,0.637105,0.289704,0.980566,...,NaN,LMGP03030018 ; LMGP03030039 ; LMGP03030062 // ...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,78758.0 ; 78779.0 ; 78802.0 // 78659.0 ; 78674...,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,GIONAPQXCTWOFO-AFPNGVDTSA-N ; LUACNSMEYWBEHO-A...,[],412 ; 413 ; 414 // 415 ; 416 ; 417
2,P4,A00004,758.56897,18.518,C18,POS,-0.501769,0.637105,0.289704,0.980566,...,NaN,LMGP10010764 ; LMGP10010273 ; LMGP10010302 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,81930.0 ; 81439.0 ; 81468.0 ; 81740.0 ; 81770....,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,QAWMHBUXVFPCLZ-SYGIZULESA-N ; LCSCBGYCNNFQBV-S...,[],418 ; 419 ; 420 ; 421 ; 422 ; 423 ; 424 ; 425 ...
3,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,http://www.hmdb.ca/metabolites/HMDB0008003 ; h...,LMGP01011483 ; LMGP01010576 ; LMGP01011787 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59451.0 ; 59420.0 ; 59709.0 ; 59741.0 ; 59483....,https://metlin.scripps.edu/metabo_info.php?mol...,52922448 ; 24778688 ; 52923053 ; 52923093 ; 24...,https://pubchem.ncbi.nlm.nih.gov/compound/5292...,LJUDFFWSOQEQKW-QCFYSATCSA-N ; AEPVLLKGEBHLEJ-K...,[],0 ; 1 ; 3 ; 12 ; 13 ; 15 ; 16 ; 17 ; 18 ; 19 ;...
4,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,NaN,LMGP03020070 ; LMGP03020087 ; LMGP03020091,http://www.lipidmaps.org/data/LMSDRecord.php?L...,78717.0 ; 78734.0 ; 78738.0,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,BSYSTFKERYSAHH-PVXQIPPMSA-N ; IQICQBYOIPVODO-P...,[],60 ; 61 ; 62
5,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,NaN,NaN,NaN,40767.0 ; 40769.0,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,NaN,[],63 ; 64
6,P1,A00001,760.58465,19.341,C18,POS,-0.414481,0.771123,-0.221210,1.181282,...,NaN,LMGP10010763 ; LMGP10010272 ; LMGP10010544 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,81929.0 ; 81438.0 ; 81710.0 ; 81467.0 ; 81739....,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,FKASCVIVTHYAIW-YIHXYWFXSA-N ; KMVWAJLBKVNELJ-B...,[],65 ; 66 ; 67 ; 68 ; 69 ; 70 ; 71 ; 72 ; 73 ; 7...
7,P3,A00003,782.56882,18.116,C18,POS,-0.137611,0.566504,-0.190560,-0.264693,...,http://www.hmdb.ca/metabolites/HMDB0008138 ; h...,LMGP01010937 ; LMGP01012195 ; LMGP01010955 ; L...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59584.0 ; 59811.0 ; 59649.0 ; 59680.0 ; 59875....,https://metlin.scripps.edu/metabo_info.php?mol...,0 ; 5288075 ; 53478951 ; 24778989 ; 52922901 ;...,https://pubchem.ncbi.nlm.nih.gov/compound/0 ; ...,FVXDQWZBHIXIEJ-LNDKUQBDSA-N ; VRXRAEQYIWJLSH-O...,[],190 ; 191 ; 192 ; 193 ; 194 ; 195 ; 196 ; 197 ...
8,P3,A00003,782.56882,18.116,C18,POS,-0.137611,0.566504,-0.190560,-0.264693,...,http://www.hmdb.ca/metabolites/HMDB0008462 ; h...,LMGP01010007 // LMGP02010815 ; LMGP02010560 ; ...,http://www.lipidmaps.org/data/LMSDRecord.php?L...,39371.0 ; 39372.0 ; 39513.0 ; 39516.0 ; 39517....,https://metlin.scr